In [1]:
import sqlite3
import pandas as pd

# Tạo kết nối SQLite trong bộ nhớ
conn = sqlite3.connect(":memory:")
cursor = conn.cursor()


# Câu 1: Tính hệ số tương quan giữa A và B bằng SQL

In [2]:
cursor.execute('''
    CREATE TABLE Scores (
        day TEXT,
        A REAL,
        B REAL
    )
''')

data = [
    ("Day 1", 8, 9),
    ("Day 2", 7.5, 8.5),
    ("Day 3", 6, 7),
    ("Day 4", 7, 6)
]

cursor.executemany("INSERT INTO Scores VALUES (?, ?, ?)", data)
conn.commit()


In [3]:
import math
conn.create_function("SQRT", 1, math.sqrt)


In [4]:
query = '''
SELECT
    (COUNT(*) * SUM(A * B) - SUM(A) * SUM(B)) /
    (SQRT(COUNT(*) * SUM(A * A) - SUM(A) * SUM(A)) *
     SQRT(COUNT(*) * SUM(B * B) - SUM(B) * SUM(B))) AS correlation_AB
FROM Scores
'''
result = cursor.execute(query).fetchone()
print("Correlation A vs B:", result[0])



Correlation A vs B: 0.6910506641398757


# Câu 2: Chuyển dữ liệu sang dạng quan hệ & kiểm định χ² bằng Python

In [5]:
cursor.execute('''
    CREATE TABLE Scores_ABC (
        day TEXT,
        A REAL,
        B REAL,
        C REAL
    )
''')

data = [
    ("Day 1", 8, 9, 7),
    ("Day 2", 7.5, 8.5, 7),
    ("Day 3", 6, 7, 8),
    ("Day 4", 7, 6, 5)
]
cursor.executemany("INSERT INTO Scores_ABC VALUES (?, ?, ?, ?)", data)
conn.commit()


In [6]:
df = pd.read_sql_query("SELECT * FROM Scores_ABC", conn)
df_long = df.melt(id_vars=["day"], var_name="model", value_name="score")
print(df_long.head())


     day model  score
0  Day 1     A    8.0
1  Day 2     A    7.5
2  Day 3     A    6.0
3  Day 4     A    7.0
4  Day 1     B    9.0


In [7]:
import scipy.stats as stats

# Phân loại điểm số (ví dụ: Low < 6, Medium 6-8, High > 8)
df_long["score_cat"] = pd.cut(df_long["score"], bins=[0, 6, 8, 10], labels=["Low", "Medium", "High"])

# Tạo bảng tần suất
contingency = pd.crosstab(df_long["model"], df_long["score_cat"])

# Chi-square test
chi2, p, _, _ = stats.chi2_contingency(contingency)
print("Chi-square:", chi2, "| p-value:", p)


Chi-square: 5.142857142857143 | p-value: 0.27295102496702883


# Câu 3: Chuyển giá trị thời gian thành định dạng chuẩn

In [8]:
cursor.execute("CREATE TABLE Flights (departure_time INTEGER)")
cursor.executemany("INSERT INTO Flights (departure_time) VALUES (?)", [(830,), (1445,), (30,), (0,)])
conn.commit()


In [9]:
query = '''
SELECT departure_time,
       printf('%02d:%02d', departure_time / 100, departure_time % 100) AS time_formatted
FROM Flights
'''
df_time = pd.read_sql_query(query, conn)
print(df_time)


   departure_time time_formatted
0             830          08:30
1            1445          14:45
2              30          00:30
3               0          00:00


# Câu 4: Tìm giá trị ngoại lệ bằng MAD

In [10]:
data = [10, 12, 13, 10, 100, 11, 12]
df = pd.DataFrame({"value": data})
median = df["value"].median()
mad = (df["value"] - median).abs().median()
threshold = 1.5 * mad

df["is_outlier"] = (df["value"] - median).abs() > threshold
print(df)


   value  is_outlier
0     10        True
1     12       False
2     13       False
3     10        True
4    100        True
5     11       False
6     12       False


# Câu 5: Kiểm tra trùng lặp theo last_name và weight

In [11]:
cursor.execute('''
    CREATE TABLE Patient (
        last_name TEXT,
        weight INTEGER,
        height INTEGER
    )
''')

data = [
    ("Nguyen", 70, 170),
    ("Tran", 60, 165),
    ("Nguyen", 70, 171),
    ("Le", 80, 180),
    ("Tran", 65, 167)
]
cursor.executemany("INSERT INTO Patient VALUES (?, ?, ?)", data)
conn.commit()


In [12]:
query = '''
SELECT *
FROM Patient
WHERE (last_name, weight) IN (
    SELECT last_name, weight
    FROM Patient
    GROUP BY last_name, weight
    HAVING COUNT(*) > 1
)
'''
df_duplicates = pd.read_sql_query(query, conn)
print(df_duplicates)


  last_name  weight  height
0    Nguyen      70     170
1    Nguyen      70     171
